In [1]:
import pandas as pd
import numpy as np

In [188]:
import glob
path = "inside_airbnb_listings/summary/"
filename= "*.csv"
keys = []
listings_dict = {}

# Get all csv files in dir and map them to a tuple with the date and dataframe
# put the dates in a list for easy iteration
for fname in glob.glob(path+filename):
    keys.append(fname)
    try:
        df = pd.read_csv(fname, encoding='ISO-8859-1', low_memory=False)
        listings_dict = {**listings_dict, **{fname:df}}
    except Exception as e:
        print(e)

In [6]:
# For all the dates get the dataframes. Then get the all the associated columns.
# Count each occurance of a column to see what columns are missing.
columns = {}
for k in keys:
    for col in listings_dict.get(k).columns:
        if col in  columns.keys():
            val = columns.get(col) + 1
            columns = {**columns, **{col:val}}
        else: 
            columns = {**columns, **{col:0}}

In [10]:
present_in_all = []     
present_in_some = []          
max_value = columns.get(max(columns, key=columns.get))
## for all columns check if they are in all (max) of the datasets
[present_in_all.append(k) if v == max_value else present_in_some.append(k)  
         for (k, v) in columns.items()]
present_in_all

['id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [11]:
# Return a dict containing Series of matched column names in DataFrames 
def match_on_column_name(name='', columns=columns):
    dict_result = {}
    for key in columns.keys():
        df = columns.get(key)
        if name in df:
            dict_result = {**dict_result, **{key:df[name]}}
    return dict_result            

In [99]:
def get_grand_mean_for_column(name='', dictionary=()):
    grand_mean = 0
    matched = match_on_column_name(name, dictionary)
    means = []
    for x in matched.keys():
        match_value = matched.get(x) 
        if 'mean' in match_value.describe():
            means.append(match_value.describe()['mean'])
        else:
            print('no mean found')
    if len(means) > 0:
        grand_mean = sum(means)/len(means)
    return grand_mean

In [13]:
def results_to_tuple(action, ls):
    result = {}
    for x in ls:
        result = {**result, **{x:action(x)}}
    return result

In [17]:
# get grand mean for input columns and return a tuple 
# containing the column-name and the grand mean
results_to_tuple(lambda x: get_grand_mean_for_column(x, listings_dict)
, [ 'price', 'reviews_per_month', 'minimum_nights', ])

{'price': 135.36038075733214, 'reviews_per_month': 1.390548330482227}

In [15]:
# join all listings
joined = pd.concat(listings_dict.values())

In [45]:
joined.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'availability_365',
       'calculated_host_listings_count', 'date', 'host_id', 'host_name', 'id',
       'last_review', 'latitude', 'listing_id', 'longitude', 'm2',
       'minimum_nights', 'name', 'neighbourhood', 'neighbourhood_group',
       'number_of_reviews', 'price', 'reviews_per_month', 'room_type',
       'price_per_reviews_per_month'],
      dtype='object')

In [121]:
def get_top_w_count(cols, df):
    result = {}
    for c in cols:
        top = df[c].describe()['top']
        freq = df[c].describe()['freq']
        result = {**result, **{c:{top:freq}}}
    return result

In [154]:
def get_mean(cols, df):
    result = {}
    for c in cols:
        mean = df[c].describe()['mean']
        result = {**result, **{c:mean}}
    return result

In [193]:
top_cols = ['host_name', 'last_review' \
            , 'name', 'neighbourhood', 'room_type']
mean_cols = ['availability_365', 'calculated_host_listings_count', \
          'm2', 'minimum_nights', 'number_of_reviews', \
          'price', 'reviews_per_month']

In [194]:
#get top and mean cols and add together to one dictionary.
joined_top_dict = get_top_w_count(top_cols, joined)
joined_mean_dict = get_mean(mean_cols, joined)
joined_mean_dict.update(joined_top_dict)

In [195]:
joined_mean_dict

{'availability_365': 117.93763404102069,
 'calculated_host_listings_count': 4.1083680987118694,
 'host_name': {'Martijn': 8085},
 'last_review': {'2017-05-28': 14090},
 'm2': 96.541949015811554,
 'minimum_nights': 2.8127705878918712,
 'name': {'Amsterdam': 1603},
 'neighbourhood': {'De Baarsjes - Oud-West': 244999},
 'number_of_reviews': 18.020541995240819,
 'price': 135.18925660120155,
 'reviews_per_month': 1.3640560230125258,
 'room_type': {'Entire home/apt': 1129049}}

In [196]:
dropped_duplicate_ids = joined.drop_duplicates('id')
#get top and mean cols and add together to one dictionary.
dropped_top_dict = get_top_w_count(top_cols, dropped_duplicate_ids)
dropped_mean_dict = get_mean(mean_cols, dropped_duplicate_ids)
dropped_mean_dict.update(dropped_top_dict)

In [197]:
dropped_mean_dict

{'availability_365': 117.96454217849909,
 'calculated_host_listings_count': 5.3554199437093928,
 'host_name': {'Sammy': 303},
 'last_review': {'2016-11-13': 435},
 'm2': nan,
 'minimum_nights': 2.7395112712733778,
 'name': {'Amsterdam': 75},
 'neighbourhood': {'De Baarsjes - Oud-West': 6159},
 'number_of_reviews': 10.499539679617014,
 'price': 134.51982008049032,
 'reviews_per_month': 1.4755028730164907,
 'room_type': {'Entire home/apt': 29891}}

In [198]:
import csv
with open(path + 'filtered_id_all_listings_summary_described.csv', 'w') as f:
    w = csv.DictWriter(f, dropped_mean_dict.keys())
    w.writeheader()
    w.writerow(dropped_mean_dict)

In [18]:
# For all rows in the dataframe, divide the price by number of bedrooms
# joined["price_per_reviews_per_month"] = joined["price"].div(joined["reviews_per_month"])

In [19]:
# for each value where the price_per_bedroom is higher than 100, return NaN 
# joined.loc[joined["price_per_reviews_per_month"] > 1e100,"price_per_reviews_per_month"] = np.nan


In [20]:
# joined["price_per_reviews_per_month"].describe()

count    1.290434e+06
mean     3.166536e+02
std      6.111420e+02
min      0.000000e+00
25%      6.299213e+01
50%      1.414286e+02
75%      3.214286e+02
max      2.200000e+04
Name: price_per_reviews_per_month, dtype: float64